# Setup

In [1]:
# Get VM CPU and R version
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}
shell_call("grep -m1 'model name' /proc/cpuinfo | awk -F': ' '{printf \" CPU Model: %s \\n \",  $2}'")
shell_call("grep 'cpu cores' /proc/cpuinfo  | awk -F': ' '{a[cores]+=$2}END{printf \"CPU Cores: %s \\n \", a[cores] }'")
shell_call("grep MemTotal /proc/meminfo | awk '{printf \"RAM: %.1fGB \\n \", $2 / 1024 / 1024}'")
shell_call("R --version | head -n 1")

 CPU Model: Intel(R) Xeon(R) CPU @ 2.20GHz 
 CPU Cores: 32 
 RAM: 51.0GB 
 R version 4.4.1 (2024-06-14) -- "Race for Your Life"

In [2]:
# Install Torch

Sys.setenv(CUDA='cpu')
install.packages("torch")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
# Test Torch installation

library(torch)
torch_rand(4)

torch_tensor
 0.0814
 0.0255
 0.6608
 0.5086
[ CPUFloatType{4} ]

In [4]:
# Install BKTR

install.packages('BKTR')

### From Github (Latest Version)
# install.packages("devtools") # if not installed
# devtools::install_github("julien-hec/BKTR", ref = "main")

# For section 4 side by side plots
install.packages('ggpubr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [5]:
# The following block is because Jupyter for R does not print until the end of
# the code block execution. So, with this command we add real time printing.
# See: https://stackoverflow.com/questions/37689694/real-time-printing-to-console-with-r-in-jupyter
trace(what = "print", where = getNamespace("base"), exit = flush.console, print = FALSE)

Tracing function "print" in package "namespace:base"



[1] "print"

# Load Libraries

In [6]:
# Code to run BKTR examples #
# Lanthier, Lei, Sun and Labbe 2023 #

library('BKTR')
# library(data.table)
# library(ggplot2)
# library('ggpubr')

# Run BKTR

In [7]:
###################################
###################################
# Section 3: BKTR regressor class #
###################################
###################################

# Input data presentation
bixi_data <- BixiData$new()
ex_locs <- c('7114 - Smith / Peel', '6435 - Victoria Hall')
ex_times <- c('2019-04-17', '2019-04-18', '2019-04-19')
# Looking at a valid spatial_positions_df
print(bixi_data$spatial_positions_df[location %in% ex_locs])
# Looking at a valid temporal_positions_df
print(bixi_data$temporal_positions_df[time %in% ex_times])
# Looking at a valid data_df
print(bixi_data$data_df[
  location %in% ex_locs & time %in% ex_times,
  c(1:4, 17)
])

# Set seed for results consistency
TSR$set_params(seed = 1)
bixi_light <- BixiData$new(is_light = TRUE)
bktr_regressor <- BKTRRegressor$new(
  data_df=bixi_light$data_df,
  spatial_positions_df=bixi_light$spatial_positions_df,
  temporal_positions_df=bixi_light$temporal_positions_df)

# Launch MCMC sampling
bktr_regressor$mcmc_sampling()


Key: <location>
               location latitude longitude
                 <char>    <num>     <num>
1: 6435 - Victoria Hall 45.48129 -73.60033
2:  7114 - Smith / Peel 45.49284 -73.55642
Key: <time>
         time time_index
       <IDat>      <int>
1: 2019-04-17          2
2: 2019-04-18          3
3: 2019-04-19          4
Key: <location, time>
               location       time nb_departure area_park  humidity
                 <char>     <IDat>        <num>     <num>     <num>
1: 6435 - Victoria Hall 2019-04-17   0.24752475 0.2254071 0.1919343
2: 6435 - Victoria Hall 2019-04-18   0.16831683 0.2254071 0.4697535
3: 6435 - Victoria Hall 2019-04-19   0.07920792 0.2254071 0.9350261
4:  7114 - Smith / Peel 2019-04-17           NA 0.0652808 0.1919343
5:  7114 - Smith / Peel 2019-04-18           NA 0.0652808 0.4697535
6:  7114 - Smith / Peel 2019-04-19           NA 0.0652808 0.9350261
[1] "Iter 1     | Elapsed Time:     0.25s | MAE:  0.0921 | RMSE:  0.1202"
[1] "Iter 2     | Elapsed Time:    